In [1]:
import json
from datetime import datetime, timedelta
from shapely.geometry import LineString, Point
import folium
import webbrowser

In [2]:
# Ścieżka do pliku JSON
file_path = r'D:\repo_git\aircraft_signals\dane\data.json'

# Lista do przechowywania wszystkich obiektów JSON
data = []

# Wczytaj dane z pliku data.json (każda linia to osobny obiekt JSON)
with open(file_path, 'r') as file:
    for line in file:
        try:
            # Usuń niepotrzebne znaki nowej linii lub inne zbędne znaki
            cleaned_line = line.strip()

            # Spróbuj parsować linię jako obiekt JSON
            obj = json.loads(cleaned_line)

            # Dodaj poprawnie sparsowany obiekt do listy danych
            data.append(obj)

        except json.JSONDecodeError as e:
            print(f"Błąd podczas parsowania JSON w linii: {cleaned_line[:50]}...")
            print(f"Komunikat błędu: {str(e)}")
            continue

In [3]:
# Lista do przechowywania identyfikatorów samolotów z potencjalną kolizją oraz czasu i położenia kolizji
collisions = []
paths = []
pitrs = []
idents = []
sorted_collisions = []

In [4]:
# Przetwarzanie każdego lotu w danych
for i in range(len(data)):
    ident1 = data[i]['ident']
    waypoints1 = data[i].get('waypoints', [])
  
    if len(waypoints1) < 2:
        continue
  
    # Tworzenie ścieżki (LineString) na podstawie waypoints
    path1 = LineString([(point['lon'], point['lat']) for point in waypoints1])
    
    # Konwertowanie pitr na obiekt datetime
    pitr1 = datetime.strptime(data[i]['pitr'], '%Y-%m-%d %H:%M:%S')
    
    # Dodanie ścieżki, pitr i ident do odpowiednich list
    paths.append(path1)
    pitrs.append(pitr1)
    idents.append(ident1)
    
    # Pętla wewnętrzna do sprawdzania kolizji z innymi lotami
    for j in range(i+1, len(data)):
        ident2 = data[j]['ident']
        waypoints2 = data[j].get('waypoints', [])
        
        if len(waypoints2) < 2:
            continue
        
        # Tworzenie ścieżki (LineString) dla drugiego lotu
        path2 = LineString([(point['lon'], point['lat']) for point in waypoints2])
        
        # Konwertowanie pitr na obiekt datetime dla drugiego lotu
        pitr2 = datetime.strptime(data[j]['pitr'], '%Y-%m-%d %H:%M:%S')
        
        # Dodanie ścieżki, pitr i ident do odpowiednich list
        paths.append(path2)
        pitrs.append(pitr2)
        idents.append(ident2)


In [5]:
# Znaleziona kolizja
found_collision = False

# Przetwarzanie ścieżek w celu znalezienia kolizji
for i in range(len(paths)):
    path1 = paths[i]
    pitr1 = pitrs[i]
    ident1 = idents[i]

    for j in range(i+1, len(paths)):
        path2 = paths[j]
        pitr2 = pitrs[j]
        ident2 = idents[j]

        #SPRAWDZENIE PRZECIEC
        if path1.intersects(path2):
            intersection_point = path1.intersection(path2)
            if intersection_point.is_empty:
                continue
            if intersection_point.geom_type == 'Point':
                intersection_lat = intersection_point.y
                intersection_lon = intersection_point.x
                collisions.append((ident1, ident2, pitr1, intersection_lat, intersection_lon, pitr2))
            break

In [6]:
for i in range(min(10, len(collisions))):
    ident1, ident2, intersection_lat, intersection_lon

    # Wydrukuj informacje o kolizji
    print(f"Collision detected between {ident1} and {ident2}")
    print(f"Intersection point coordinates: ({intersection_lat:.2f}, {intersection_lon:.2f})")
    print("-------------------------------")

Collision detected between SKW3487 and SKW3487
Intersection point coordinates: (38.24, -120.76)
-------------------------------
Collision detected between SKW3487 and SKW3487
Intersection point coordinates: (38.24, -120.76)
-------------------------------
Collision detected between SKW3487 and SKW3487
Intersection point coordinates: (38.24, -120.76)
-------------------------------
Collision detected between SKW3487 and SKW3487
Intersection point coordinates: (38.24, -120.76)
-------------------------------
Collision detected between SKW3487 and SKW3487
Intersection point coordinates: (38.24, -120.76)
-------------------------------
Collision detected between SKW3487 and SKW3487
Intersection point coordinates: (38.24, -120.76)
-------------------------------
Collision detected between SKW3487 and SKW3487
Intersection point coordinates: (38.24, -120.76)
-------------------------------
Collision detected between SKW3487 and SKW3487
Intersection point coordinates: (38.24, -120.76)
--------

In [7]:
# Paleta kolorów
colors = [ 'green', 'pink', 'blue', 'purple', 'darkred' , 'red', 'orange', 'darkblue']

# Sortowanie listy collisions po czasie pitr1
collisions.sort(key=lambda x: x[2])

# Stwórz mapę
mymap = folium.Map(location=[0, 0], zoom_start=2)

# Zdefiniuj unikalne kolory dla każdego samolotu
plane_colors = {}

# Dodaj trajektorie samolotów z potencjalną kolizją do mapy
for idx in range(min(7, len(collisions))):
    if idx == 2:
        continue  # Pomijamy trzecią kolizję
    collision = collisions[idx]
    ident1, ident2, pitr1, intersection_lat, intersection_lon, pitr2 = collision
    
    # Pobierz indeksy tras dla ident1 i ident2
    index1 = idents.index(ident1)
    index2 = idents.index(ident2)
    
    # Pobierz waypoints i kolor dla ident1
    waypoints1 = data[index1].get('waypoints', [])
    if len(waypoints1) < 2:
        # Jeśli są tylko punkty początkowy i końcowy, dodaj punkty pośrednie
        start_point = waypoints1[0]
        end_point = waypoints1[-1]
        waypoints1 = [start_point, end_point]
    color1 = plane_colors.setdefault(ident1, colors[len(plane_colors) % len(colors)])
    
    # Pobierz waypoints i kolor dla ident2
    waypoints2 = data[index2].get('waypoints', [])
    if len(waypoints2) < 2:
        # Jeśli są tylko punkty początkowy i końcowy, dodaj punkty pośrednie
        start_point = waypoints2[0]
        end_point = waypoints2[-1]
        waypoints2 = [start_point, end_point]
    color2 = plane_colors.setdefault(ident2, colors[len(plane_colors) % len(colors)])
    
    # Dodaj trajektorię dla ident1 do mapy
    folium.PolyLine([(point['lat'], point['lon']) for point in waypoints1], color=color1, weight=2.5, opacity=1).add_to(mymap)
    
    # Dodaj trajektorię dla ident2 do mapy
    folium.PolyLine([(point['lat'], point['lon']) for point in waypoints2], color=color2, weight=2.5, opacity=1).add_to(mymap)
    
    # Dodaj marker dla początkowego punktu trasy (pierwszy punkt)
    if waypoints1:
        folium.Marker(location=(waypoints1[0]['lat'], waypoints1[0]['lon']), icon=folium.Icon(color=color1), popup=f"Początkowy punkt: {ident1}").add_to(mymap)
    
    # Dodaj marker dla końcowego punktu trasy (ostatni punkt)
    if waypoints1:
        folium.Marker(location=(waypoints1[-1]['lat'], waypoints1[-1]['lon']), icon=folium.Icon(color=color1), popup=f"Końcowy punkt: {ident1}").add_to(mymap)
    
    # Dodaj marker dla początkowego punktu trasy (pierwszy punkt)
    if waypoints2:
        folium.Marker(location=(waypoints2[0]['lat'], waypoints2[0]['lon']), icon=folium.Icon(color=color2), popup=f"Początkowy punkt: {ident2}").add_to(mymap)
    
    # Dodaj marker dla końcowego punktu trasy (ostatni punkt)
    if waypoints2:
        folium.Marker(location=(waypoints2[-1]['lat'], waypoints2[-1]['lon']), icon=folium.Icon(color=color2), popup=f"Końcowy punkt: {ident2}").add_to(mymap)
    
    # Dodaj marker dla punktu przecięcia (kolizji)
    folium.Marker(location=(intersection_lat, intersection_lon), icon=folium.Icon(color='black'), popup=f"Potencjalna kolizja: {ident1} i {ident2}\nPołożenie: {intersection_lat:.2f}, {intersection_lon:.2f}").add_to(mymap)

# Zapisz mapę do pliku HTML
mymap.save('intersecting_flights_map.html')

print("Mapa została zapisana jako intersecting_flights_map.html")

Mapa została zapisana jako intersecting_flights_map.html
